# ACKNOWLEDGMENT
## This exercise is prepared using the book 'Learning Geospatial Analysis with Python. 3rd Edition. Author: Joel Lawhead'

# Activity 01
This activity creates a simple GIS using python's built-in module called 'turtle'. If new to turtle, please read more about it before here, https://docs.python.org/3/library/turtle.html, diving into the rest of the activity.

In [1]:
# import the required python library i.e., turtle

import turtle as t

## set up data model


create constant variables; i.e., assign constants to some common elements that are used for All cities will have a name, one or more points, and a population count

In [6]:
NAME=0
POINTS=1
POP=2

Let's set up the data for Albury as a list with a name, polygon points,
and population (as of 2020). Note that the coordinates are a list within a list

In [22]:
state=["NSW",[[141,-34], [141,-29],[154,-28],[150,-38]],8172500 ]


Now let's add a few cities of NSW. Each city's location consists of a single
point as a longitude and latitude pair. These entries will complete our GIS data
model. Start with an empty list called cities and then append the data to
this list for each city.

In [23]:
cities=[]
cities.append(["SYDNEY",[151.21, -33.87], 5231150])
cities.append(["ALBURY",[146.91, -36.07], 56036])
cities.append(["ORANGE", [149.10, -33.28],40493])

## Prep for map rendering 

First, define a map size

In [24]:
map_width=400
map_height=300

In order to scale the map to the graphics canvas, we must first determine the
bounding box of the largest layer, which is the state. We'll set the map's
bounding box to a global scale and reduce it to the size of the state. To do so,
we'll loop through the longitude and latitude of each point and compare it with
the current minimum and maximum x and y values. If it is larger than the
current maximum or smaller than the current minimum, we'll make this value
the new maximum or minimum, respectively:

In [25]:
minx = 180
maxx = -180
miny = 90
maxy = -90
for x,y in state[POINTS]:
    if x < minx:
        minx = x
    elif x > maxx:
        maxx = x
    if y < miny:
        miny = y
    elif y > maxy:
        maxy = y

The second step when it comes to scaling is calculating a ratio between the actual
state and the tiny canvas that we will render it on. This ratio is used for
coordinate to pixel conversion. We get the size along the x and y axes of the state
and then we divide the map width and height by these numbers to get our
scaling ratio:

In [26]:
dist_x = maxx - minx
dist_y = maxy - miny
x_ratio = map_width / dist_x
y_ratio = map_height / dist_y

## convert a point of map coordinates to pixel coordinates
We'll create a user-defined function called convert(), to transform a point in the map coordinates from one of our data layers into pixel coordinates using the previous calculations. You'll notice that, in the end, we divide the map width and height in half and subtract it from the final conversion to account for the unusual center origin of the turtle graphics canvas. Every
geospatial program has some form of this function:

In [27]:
def convert(point):
    lon=point[0]
    lat=point[1]
    x = map_width - ((maxx - lon) * x_ratio)
    y = map_height - ((maxy - lat) * y_ratio)
    # Python turtle graphics start in the
    # middle of the screen so we must offset the points so they are centered
    x = x - (map_width/2)
    y = y - (map_height/2)
    return [x,y]

## Render the map

The turtle module uses the concept of a cursor, known as a pen. Moving the cursor around the canvas is exactly the same as moving a pen around a piece of paper. The cursor will draw a line when you move it. You'll notice that, throughout the code, we use the t.up() and t.down() commands to pick the pen up when we want to move to a new location and put it down when we're ready to draw. We have some important steps to follow in this section, so let's get started.

#### Since the border of NSW is a polygon, we must draw a line between the last point and the first point to close the polygon. We can also leave out the closing step and just add a duplicate point to the NSW dataset. Once we've drawn the state, we'll use the write() method to label the polygon

In [29]:
t.up()
first_pixel=None
for point in state[POINTS]:
    pixel=convert(point)
    if not first_pixel:
        first_pixel=pixel
    t.goto(pixel)
    t.down()
t.goto(first_pixel)
t.up()
t.goto([0,0])
t.write(state[NAME], align="center", font=("Arial", 16,"bold"))

Now, we'll render the cities as point locations and label them with their names
and population. Since the cities are a group of features in a list, we'll loop
through them to render them. Instead of drawing lines by moving the pen
around, we'll use the turtle dot() method to plot a small circle at the pixel
coordinate that's returned by our basicGISconvert() function. We'll then
label the dot with the city's name and add the population. You'll notice that we
must convert the population number into a string in order to use it in the turtle
write() method. To do so, we will use Python's built-in str() function:

In [30]:
for city in cities:
    pixel=convert(city[POINTS])
    t.up()
    t.goto(pixel)
    
    #place a point for the city
    t.dot(10)
    
    #label the city
    t.write(city[NAME] + ", Pop.: " + str(city[POP]), align="left")
    t.up()
    

Now, we will perform one last operation to prove that we have created a real
GIS. We will perform an attribute query on our data to determine which city has
the largest population. Then, we'll perform a spatial query to see which city lies
the furthest west. Finally, we'll print the answers to our questions on our
thematic map page safely, out of the range of the map.

For our query engine, we'll use Python's built-in min() and max() functions.
These functions take a list as an argument and return the minimum and
maximum values of this list. These functions have a special feature called a key
argument that allows you to sort complex objects. Since we are dealing with
nested lists in our data model, we'll take advantage of the key argument in these
functions. The key argument accepts a function that temporarily alters the list for
evaluation before a final value is returned. In this case, we want to isolate the
population values for comparison, and then the points. We could write a whole
new function to return the specified value, but we can use Python's lambda
keyword instead. The lambda keyword defines an anonymous function that is
used inline. Other Python functions can be used inline, such as the string
function, str(), but they are not anonymous. This temporary function will
isolate our value of interest.

#### which city has the largest population?

In [31]:
biggest_city = max(cities, key=lambda city:city[POP])
t.goto(0,-200)
t.write("The biggest city is: " + biggest_city[NAME])

#### which city lies the furthest west?

In [32]:
western_city = min(cities, key=lambda city:city[POINTS])
t.goto(0,-220)
t.write("The western-most city is: " + western_city[NAME])

The last two commands are just for cleanup purposes. First, we hide the cursor.
Then, we call the turtle done() method, which will keep the turtle graphics
window with our map on it open until we choose to close it using the close
handle at the top of the window:

In [33]:
t.pen(shown=False)
t.done()